In [1]:
!apt install libgl1-mesa-glx -y
!pip install opencv-python
!pip install scikit-learn
!pip install matplotlib
!pip install tensorflow
!pip install pandas
!pip install seaborn
!pip install gdown

import gdown
import cv2
import numpy as np
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt


FILE_ID = '1L-yD_x0hSRX3UMEGv_qDcp8Jk5WWFKlz'
url = f"https://drive.google.com/uc?id={FILE_ID}"
gdown.download(url, "file.zip", quiet=False)
!unzip file.zip
!rm file.zip

%cd EfficientNet
!git clone https://github.com/gsrinivas37/generate_tf_records.git
%cd generate_tf_records
!rm -rf '../train_data/.DS_Store'
!rm -rf '../test_data/.DS_Store'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0 libglx-mesa0 libglx0
  libllvm12 libpciaccess0 libsensors-config libsensors5 libvulkan1
  libwayland-client0 libx11-6 libx11-data libx11-xcb1 libxau6 libxcb-dri2-0
  libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-randr0 libxcb-shm0
  libxcb-sync1 libxcb-xfixes0 libxcb1 libxdmcp6 libxext6 libxfixes3
  libxshmfence1 libxxf86vm1 mesa-vulkan-drivers
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libgl1 libgl1-mesa-dri libgl1-mesa-glx libglapi-mesa libglvnd0
  libglx-mesa0 libglx0 libllvm12 libpciaccess0 libsensors-config libsensors5
  libvulkan1 libwayland-client0 libx11-6 libx11-data l

2023-04-26 04:01:44.589421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Downloading...
From (uriginal): https://drive.google.com/uc?id=1L-yD_x0hSRX3UMEGv_qDcp8Jk5WWFKlz
From (redirected): https://drive.google.com/uc?id=1L-yD_x0hSRX3UMEGv_qDcp8Jk5WWFKlz&confirm=t&uuid=41d5cba2-4e84-4e82-bd17-e32bbf312bbf
To: /root/file.zip
100%|██████████| 182M/182M [00:06<00:00, 26.1MB/s] 


Archive:  file.zip
   creating: EfficientNet/
  inflating: EfficientNet/.DS_Store  
  inflating: __MACOSX/EfficientNet/._.DS_Store  
   creating: EfficientNet/test_data/
  inflating: __MACOSX/EfficientNet/._test_data  
   creating: EfficientNet/train_data/
  inflating: __MACOSX/EfficientNet/._train_data  
  inflating: EfficientNet/test_data/.DS_Store  
  inflating: __MACOSX/EfficientNet/test_data/._.DS_Store  
   creating: EfficientNet/test_data/Fly/
  inflating: __MACOSX/EfficientNet/test_data/._Fly  
   creating: EfficientNet/test_data/Bees_Wasp/
  inflating: __MACOSX/EfficientNet/test_data/._Bees_Wasp  
   creating: EfficientNet/test_data/Bumble_Bee/
  inflating: __MACOSX/EfficientNet/test_data/._Bumble_Bee  
   creating: EfficientNet/test_data/Other/
  inflating: __MACOSX/EfficientNet/test_data/._Other  
   creating: EfficientNet/test_data/Hummingbird/
  inflating: __MACOSX/EfficientNet/test_data/._Hummingbird  
   creating: EfficientNet/test_data/Butterflies_Moths/
  inflating: __

In [2]:
from tensorflow.keras.applications import EfficientNetB1 as EffNetModel
IMG_SIZE = 240

In [3]:
%cd /root/EfficientNet/generate_tf_records
from prepare_efficientnet_data import *

NUM_CLASSES = 6

train_x, train_y = get_data('../train_data', IMG_SIZE)
test_x, test_y = get_data('../test_data', IMG_SIZE)

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EffNetModel(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.15
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-14:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()    


/root/EfficientNet/generate_tf_records
Labels
Butterflies_Moths    352
Bees_Wasp            291
Fly                  225
Other                174
Bumble_Bee           132
Hummingbird           17
Name: count, dtype: int64


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Labels
Butterflies_Moths    107
Fly                   61
Bees_Wasp             49
Other                 43
Bumble_Bee            31
Hummingbird            7
Name: count, dtype: int64


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-04-26 04:02:12.031336: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10186 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:04:00.0, compute capability: 8.6


In [ ]:
epochs = 10000 

model = build_model(num_classes=NUM_CLASSES)
unfreeze_model(model)
# hist = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=epochs, verbose=2)
hist = model.fit(train_x, train_y, batch_size=32, epochs=epochs, verbose=2)
plot_hist(hist)

27018416/27018416 [==============================] - 2s 0us/step
Epoch 1/10000


2023-04-26 04:02:31.605926: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNet/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-26 04:02:34.606495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-26 04:02:35.721901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-26 04:02:36.218834: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f2a0834fe30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-26 04:02:36.218868: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-04-26 04:02:36.224966: I tensorflow/compiler/mlir/tensorflow/utils/dump_m

38/38 - 21s - loss: 1.9399 - accuracy: 0.1864 - 21s/epoch - 544ms/step
Epoch 2/10000
38/38 - 3s - loss: 1.8702 - accuracy: 0.2292 - 3s/epoch - 74ms/step
Epoch 3/10000
38/38 - 3s - loss: 1.8228 - accuracy: 0.2418 - 3s/epoch - 74ms/step
Epoch 4/10000
38/38 - 3s - loss: 1.7854 - accuracy: 0.2662 - 3s/epoch - 74ms/step
Epoch 5/10000
38/38 - 3s - loss: 1.7791 - accuracy: 0.2477 - 3s/epoch - 74ms/step
Epoch 6/10000
38/38 - 3s - loss: 1.7651 - accuracy: 0.2494 - 3s/epoch - 74ms/step
Epoch 7/10000
38/38 - 3s - loss: 1.7473 - accuracy: 0.2611 - 3s/epoch - 74ms/step
Epoch 8/10000
38/38 - 3s - loss: 1.7434 - accuracy: 0.2704 - 3s/epoch - 74ms/step
Epoch 9/10000
38/38 - 3s - loss: 1.7348 - accuracy: 0.2695 - 3s/epoch - 75ms/step
Epoch 10/10000
38/38 - 3s - loss: 1.7066 - accuracy: 0.2888 - 3s/epoch - 74ms/step
Epoch 11/10000
38/38 - 3s - loss: 1.7087 - accuracy: 0.2695 - 3s/epoch - 74ms/step
Epoch 12/10000
38/38 - 3s - loss: 1.7292 - accuracy: 0.2737 - 3s/epoch - 74ms/step
Epoch 13/10000
38/38 - 3

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
def predict(img_path = 'unseen.jfif'):
  img = cv2.imread(img_path)
  img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

  x = np.expand_dims(img, axis=0)
  x = preprocess_input(x)

  print('Input image shape:', x.shape)

  my_image = imread(img_path)
  imshow(my_image)

  preds=model.predict(x)
  print("predicted class: ", preds)

predict('/content/EfficientNet/test_data/Butterflies_Moths/motion_2021-06-17_22_39_36_172_mp4-112_jpg.rf.928efb71e393fdc636d09c6d8c43e8d7.jpg')